In [ ]:
import cv2
import numpy as np
from tkinter import *
from PIL import Image,ImageTk
from tkinter import filedialog

minimum_accuracy = 0.4
width = 500
image_file = "C:/Users/msi/Desktop/openCV project/test_photos/Air jordan 1 high og dior 166.jpg"
title = "shoes recognition program"

def ShoesDetection(image):
    net = cv2.dnn.readNet('./my_weight/my_yolo_2000.weights','./custom/my_yolo.cfg')
    classes = []
    layers = []
    scalesize = 0.00392 #1을 255로 나눈 값 반올림
    minimum_accuracy = float(accuracySpin.get())
    with open('./custom/my_classes.names', "r") as f:
        for r in f.readlines():
            classes.append(r.strip())
    layerNames = net.getLayerNames()
    for i in net.getUnconnectedOutLayers():
        layers.append(layerNames[i[0] - 1])
    h, w = image.shape[:2]
    height = int(h * (width /w))
    img = cv2.resize(image,(width, height))
    blobImage = cv2.dnn.blobFromImage(img, scalesize, (416, 416), swapRB = True, crop = False)
    net.setInput(blobImage)
    outs = net.forward(layers)
    locationbox = []
    namebox = []
    accuracy = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:] 
            class_id = np.argmax(scores) #scores에서 가장 높은 값을 해당 물체로 인식
            confidence = scores[class_id]
            if confidence > minimum_accuracy:
                #detection안의 값은 상대위치이므로 width와 height을 곱해줌.
                centerX = int(detection[0] * width)
                centerY = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(centerX - w/2)
                y = int(centerY - h/2)
                locationbox.append([x, y, w, h])
                accuracy.append(float(confidence))
                namebox.append(classes[class_id])
    #노이즈 제거
    indexes = cv2.dnn.NMSBoxes(locationbox, accuracy, minimum_accuracy, 0.4)
    for i in range(len(locationbox)):
        if i in indexes:
            x, y, w, h = locationbox[i]
            label = '{} {:,.2%}'.format(namebox[i], accuracy[i])
            #신발마다 색깔 지정(에어 조던: 빨강, 척: 파랑, 독일군:초록)
            if namebox[i] == 'Air jordan 1 high og dior':
                color = (0,0,255)
            elif namebox[i] == 'Chuck 70 Classic Black 162058C':
                color = (255, 0, 0)
            elif namebox[i] == 'German Army Trainers':
                color = (0, 255, 0)
            cv2.rectangle(img,(x,y),(x + w, y + h),color, 2)
            #글씨가 위에 닿지 않도록하고 사각형 중간에 오도록
            startY = y - 10 if y - 10 > 10 else y + 10
            startX = int(x - w/2)
            cv2.putText(img, label, (startX, startY), cv2.FONT_ITALIC, 0.5, (0,0,0), 2)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    imgtk = ImageTk.PhotoImage(image=image)
    detection_image.config(image=imgtk)
    detection_image.image = imgtk

def FileOpen():
    file = filedialog.askopenfilename(initialdir = "./",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
    image_file = cv2.imread(file)
    assert not isinstance(image_file,type(None))
    ShoesDetection(image_file)
    
root = Tk()
#root =Toplevel() #Tk() 자체에서 오류가 날 경우 root = Toplevel()로 실행시킨 뒤 다시 Tk()로 작동시키면 정상작동됩니다.
root.title(title)
root.geometry()

read_image = cv2.imread(image_file)
image = cv2.cvtColor(read_image, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
imgtk = ImageTk.PhotoImage(image=image)

label=Label(root, text=title)
label.config(font=("Courier", 15))
label.grid(row=0,column=0,columnspan=4)

accuracyLabel = Label(root, text = "minimum accuracy: ")
accuracyLabel.grid(row=1, column= 0)
accuracyIntVar = IntVar(value = minimum_accuracy)
accuracySpin = Spinbox(root, textvariable = accuracyIntVar, from_= 0, to = 1, increment = 0.05, justify = RIGHT)
accuracySpin.grid(row=1, column = 1)

Button(root,text="File Select", height=2,command=lambda:FileOpen()).grid(row=1, column=2, columnspan=2, sticky=(W, E))

detection_image=Label(root, image = imgtk)
detection_image.grid(row=2,column=0,columnspan=3)

ShoesDetection(read_image)

root.mainloop()